### MechInterp

In [ ]:
softmax(x@W_Q@W_K@x.T)@x@W_V

In [ ]:
_, cache = model.run_with_cache(tokens)

In [ ]:
accumulated_resid = cache.accumulated_resid(
    layer=-1,
    pos_slice=-1
)

In [ ]:
correct_tokens, incorrect_tokens = answer_tokens.unbind()

In [ ]:
def logit_difference(logit):
    return logit[correct_token] - logit[incorrect_token]

In [ ]:
prob(0) = sigmoid(logit0 - logit1)

In [ ]:
logit0 = final_resid @ W_U[0]

logit1 = final_resid @ W_U[1]

In [ ]:
cache["", ]

In [ ]:
v_2@W_E@W_Q@W_K@[v_0, v_1]

In [ ]:
gmae mode, record activations, for each cell, empty, white, black

In [ ]:
_, cache = model.run_with_cache(tokens)

In [1]:
from transformer_lens.utils import get_act_name

In [6]:
input_name = get_act_name("resid_post", 2)

In [7]:
input_name

'blocks.2.hook_resid_post'

In [8]:
output_name = get_act_name("normalized")

In [ ]:
input_activations = cache[input_name]

In [ ]:
output_activations = cache[output_name]

In [ ]:
features are linear representations
features are represented as directions

In [9]:
from einops import einsum

In [ ]:
interference = einsum(
    W, W, "instances feature1 dim, instances feature2 dim -> instance feature1 feature2"
)

In [ ]:
step 1: x = embed + pos_embed + sum(12 heads)
step 2: q = x@W_Q, k=x@W_K
step 3: q = [embed+pos_embed+sum(12 heads)]@W_Q

k = [embed+pos_embed+sum(12 heads)]@W_K
step 4:  

In [ ]:
logits = final_ln @ W_U

final_ln = 

rank, world_size, tensor_parallel_size, pipeline_parallel_size, data_parallel_size

In [ ]:
tensor.storage().resize_(0)

In [10]:
import torch
from torch import nn
import torch.nn.functional as F

In [11]:
class GreedySharding:
    def __init__(self, module, parallel_context):
        self.module = module
        self.parallel_context = parallel_context
    
    def shard(self):
        self._shard_parameters()
        return self.module
    
    def _shard_parameters(self):
        module = self.module
        
        for param in module.parameters():
            assert not hasattr(module, "_is_sharded")
            orig_data = param.data
            shard = self._get_shard(param.data)
            free_storage(orig_data)
            param.data = shard
    
    def _get_shard(self, data):
        parallel_context = self.parallel_context
        world_size = parallel_context.get_world_size()
        rank = parallel_context.get_rank()
        
        chunks = list(data.flatten().chunks(world_size))
        
        while len(chunks) < world_size:
            chunks.append(torch.empty(1))
            
        shard = chunks[rank]
        num_elements = chunks[rank].numel()
        num_to_pad = num_elements - shard.numel()
        
        if num_to_pad > 0:
            shard = F.pad(shard, num_to_pad)
        
        return shard

In [13]:
import hmac
import hashlib

In [14]:
def compute_digest(key, serialized_message):
    return hmac.new(key, serialized_message).digest()

In [ ]:
clock cycle 1: F(0, 0)
clock cycle 2: F(1, 0), F(0, 1)
clock cycle 3: F(2, 0), F(1, 1)
clock cycle 4: F(2, 1)

In [ ]:
step 1: replicate model
step 2: split a mini-batch into microbatches
step 3: forward and backward
step 4: average the gradients
step 5: update

In [ ]:
step 1: get the global rank fo the current process
step 2: resize embedding vocab size
step 3: parallelize embedding layer, linear layer, attention, and layer norm
step 4: resize the output size

In [ ]:
step 1: determine a list of global rank in that parallel group
step 2: check whether the process's global rank in that list
step 3: if yes, init a distributed group
step 4: determine local rank
step 5: save

In [15]:
class Copy(torch.autograd.Function):
    @staticmethod
    def forward(ctx, prev_stream, next_stream, input):
        ctx.prev_stream = prev_stream
        ctx.next_stream = next_stream
        
        compute_stream = torch.cuda.default_stream(
            next_stream.device
        )
        
        with torch.cuda.stream(prev_stream), torch.cuda.stream(next_stream):
            moved_input = input.to(next_stream.device)
            input.record_stream(prev_stream)
            moved_input.record_stream(compute_stream)
        
        return moved_input

    @staticmethod
    def backward(ctx, grad_input):
        prev_stream = ctx.prev_stream
        next_stream = ctx.next_stream
        
        compute_stream = torch.cuda.default_stream(prev_stream.device)
        
        with torch.cuda.stream(prev_stream), torch.cuda.stream(next_stream):
            moved_grad_input = grad_input.to(prev_stream.device)
            grad_input.record_stream(next_stream)
            moved_grad_input.record_stream(prev_stream)
        return moved_grad_input

In [ ]:
step 1: 
step 2: bind
step 3: accept
step 4: communicate
step 5: close connection

In [ ]:
class Checkpoint(torch.autograd.Function):
    @staticmethod
    def forward(ctx, phony, recomputed, function, input):
        ctx.recomputed = recomputed
        
        with torch.no_grad():
            output = function(input)
        
        return output

    @staticmethod
    def backward(ctx, grad_input):
        output, input_leaf = ctx.recomputed.pop()
        
        with torch.enable_grad():
            torch.autograd.backward(output, input_leaf)
        
        return None, None, None, input_leaf.grad

In [ ]:
corrected features vs non-correlated features
adding relu
vary sparity
vary feature importance

In [ ]:
step 1: logit_diff = logit0 - logit1
step 2: logit0 = final_ln @ W_U[0], logit1 = final_ln @ W_U[1]
step 3: logit0 - logit1 = final_ln @ W_U[0] - final_ln @ W_U[1]

= final_ln @ (W_U[0] - W_U1[])

In [ ]:
lru_cache

In [16]:
from functools import reduce

In [ ]:
friction, applied force, normal, air, gravity

In [ ]:
conditioned, stimulus-evoked, population decoding, motor imagery,

In [17]:
def add(module, inp, out):
    add_log(module, out)

In [18]:
def add_hook(model):
    for layer in model.modules():
        layer.register_forward_hook(add)

In [ ]:
stream = torch.cuda.stream(device)

In [ ]:
with torch.cuda.device(device):
    with torch.cuda.stream(stream):
        mean = xs.mean()

In [ ]:
int* a;
int* b;

In [ ]:
size_t bytes = size(int) * n

In [ ]:
a = (int*)malloc(bytes)

In [ ]:
b = (int*)malloc(bytes)

In [ ]:
minimize communication
maximize memory storage
minimize FLOPs

In [ ]:
step 1: rank
step 2: resize vocab embedding
step 3: parallelize embedding, linear, attention, layer norm
step 4: resize lm_head

In [19]:
from einops import rearrange

In [ ]:
def compute_loss(model xb, yb):
    logits = model(xb)
    
    logits = rearrange(preds, "bs seq_len n_emed -> (bs seq_len) n_embed")
    yb = rearrange(yb, "bs seq_len -> (bs seq_len)")
    
    loss = F.cross_entropy(logits, yb)

In [ ]:
message passing, p2p, 

In [20]:
from torch.utils.data import Dataset

In [ ]:
class CachedDataset(Dataset):
    def __init__(self, filename):
        super().__init__()
        self.filename = filename
        self.data = None
        self.cached_index = {}
    
    def prefetch(self, idxs):
        if all([for i in idxs]): # wrong
            return
    
        if self.data is None:
            self.data = torch.load(self.filename)
        
        size = sum([self.data[i].numel() for i in idxs]) * size
        
        

In [21]:
class BatchNorm(nn.Module):
    def __init__(self, mom, eps):
        super().__init__()
        self.mom, self.eps = mom, eps
        self.mean = nn.Parameter(torch.zeros(1))
        self.var = nn.Parameter(torch.ones(1))
        self.add = nn.Parameter(torch.zeros(1))
        self.mult = nn.Parameter(torch.ones(1))
        
    def update_stats(self, x):
        mean = x.mean(dim=0, keepdim=True)
        var = x.var(dim=0, keepdim=True)
        
        self.mean.lerp_(mean, self.mom)
        self.var.lerp_(var, self.mom)
        return mean, var
    
    def forward(self, x):
        with torch.no_grad():
            mean, var = self.update_stats(x)
            
        x = x-mean/(var+self.eps).sqrt()
        x = self.add + self.mult * x
        return x

In [22]:
from einops import rearrange

In [ ]:
output = rearrange(
    images,
    "bs c (n_height p_height) (n_width p_width)\
    -> bs (n_height n_width) (p_height p_width c)"
)

In [23]:
import torch

In [24]:
torch.distributed.get_backend()

RuntimeError: Default process group has not been initialized, please make sure to call init_process_group.

In [25]:
x = torch.tensor([1., 2., 3.])

In [26]:
y = torch.tensor([0.1, 0.2, 0.3])

In [27]:
x*y

tensor([0.1000, 0.4000, 0.9000])